Import packages:

In [2]:
from finetune import Classifier, config
import tensorflow as tf
import csv
import pandas
import numpy as np

Get data:

In [3]:
filePath = "Bloomington_311_processed_3.csv"
data3 = pandas.read_csv(filePath)
print(data3.shape)
print(data3.loc[82480])

(82484, 5)
service_name                                                 Yard Waste
description           Neighbor water buckets\n\nThe owner of the hou...
agency_responsible                                                  NaN
OurLabel                                                     Yard Waste
OurNumericLabel                                                       8
Name: 82480, dtype: object


Filter data only to ones with at least 40 characters and at most 1024

In [4]:
mask = (data3['description'].str.len() >=40) & (data3['description'].str.len() <=1024)
dataFiltered = data3.loc[mask]
print(dataFiltered.shape)

(42189, 5)


Check for columns with NaN values

In [5]:
dataFiltered.columns[dataFiltered.isna().any()].tolist()
# ourLabel doesn't have NaN values, so that is good. 

['agency_responsible']

Prepare training data

In [6]:
trainingData = dataFiltered[["description", "OurLabel"]]
print(trainingData.shape)
print(trainingData.loc[0])
# bigMask = (trainingData["description"].str.len() >=1000)
# print(trainingData.loc[bigMask].shape)
# Split in train and test 80/20
msk = np.random.rand(len(trainingData)) < 0.8
train = trainingData[msk]
test = trainingData[~msk]

print(train.shape)
print(test.shape)

(42189, 2)
description    Owner is in the hospital and has been for awhi...
OurLabel                                          Animal Control
Name: 0, dtype: object
(33746, 2)
(8443, 2)


Training a model with finetune (this part fails because of CPU overload)

In [ ]:
model = Classifier(max_length=1024)               # Load base model
model.fit(train["description"], train["OurLabel"])          # Finetune base model on custom data

Instructions for updating:
Shapes are always computed; don't use the compute_shapes as it has no effect.


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:108: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Instructions for updating:
Use `tf.variables_initializer` instead.


Predictions

In [ ]:
predictions = model.predict(test) # [{'class_1': 0.23, 'class_2': 0.54, ..}, ..]
model.save("newModel")                   # Serialize the model to disk
